In [1]:
import pandas as pd
import numpy as np

# Load and process data

In [2]:
!ls ./datas/ > cours.txt

In [3]:
# For now stock prices are manually downloaded on Yahoo Finance --> TO BE IMPROVED
files_paths = []

with open("cours.txt", "r") as file:
    for line in file:
        files_paths.append("./datas/" + line[:-1])

In [4]:
# Create a dataframe with stock prices with a common time period

stock_prices_df = pd.DataFrame(columns=["Date"])

for file in files_paths:
    ticker = file.split("/")[2][:-4] # Extract ticker name from the file name
    df = pd.read_csv(file)
    df = df[["Date", "Close"]]
    df.columns = ["Date", ticker]
    stock_prices_df = pd.merge(stock_prices_df, df, how='outer', on='Date')

stock_prices_df = stock_prices_df.sort_values(by="Date")
stock_prices_df.index = stock_prices_df["Date"]
stock_prices_df.drop("Date", axis=1, inplace=True)
stock_prices_df.dropna(inplace=True)

In [5]:
# Select desired time_period --> TO BE IMPROVED
nb_days = 250
stock_prices_df = stock_prices_df.iloc[-nb_days:, :]

In [6]:
# Compute daily returns (continuous)

for col in stock_prices_df.columns:
    stock_prices_df[col] = np.log(stock_prices_df[col].shift(1) / stock_prices_df[col]) # Dividends not included --> TO BE IMPROVED
    
stock_prices_df.dropna(inplace=True)
market_prices_df = stock_prices_df["^FCHI"]
stock_prices_df.drop("^FCHI", axis=1, inplace=True)

# Compute optimal portfolio allocations

In [58]:
def compute_GMVP_weights(inv_covariance_matrix, unit_vector):
    """ Compute GMVP (Global minimum variance portfolio)"""
    
    return (inv_covariance_matrix @ unit_vector) / (unit_vector.T @ inv_covariance_matrix @ unit_vector)

In [68]:
def compute_efficient_portfolio_weights(inv_covariance_matrix, unit_vector, returns_vector, target_return):
    
    returns_unit = np.c_[returns_vector, unit_vector].T
    
    a = returns_unit @ inv_covariance_matrix @ returns_unit.T
    inv_a = np.linalg.inv(a)

    transfo_matrix = inv_covariance_matrix @ returns_unit.T @ inv_a
    
    return transfo_matrix @ np.array([target_return, 1]).reshape(2, 1)

In [43]:
def compute_metrics_ptf(weights, returns, covariance):
    
    expected_return = weights.T @ returns
    expected_var = weights.T @ covariance @ weights
    expected_volat = np.sqrt(expected_var)
    
    expected_return = expected_return[0][0]
    expected_var = expected_var.values[0][0]
    expected_volat = expected_volat.values[0][0]
    
    print("expected_return : {:.6f}".format(expected_return))
    print("expected_variance : {:.6f}".format(expected_var))
    print("expected_volatility : {:.6f}".format(expected_volat))
    
    return expected_return, expected_var, expected_volat    

In [63]:
# Compute returns for the period, covariance matrix and unit vector

returns = stock_prices_df.sum() # Mean for average daily returns or sum for period return. To rebase on a yearly return ?
returns_vector = np.array(returns).reshape(len(returns), 1)

covariance_matrix = stock_prices_df.cov()
inv_covariance_matrix = np.linalg.inv(covariance_matrix)

unit_vector = np.full((len(returns), 1), 1)

In [60]:
# Compute GMVP (Global minimum variance portfolio)

weights_GMVP = compute_GMVP_weights(inv_covariance_matrix, unit_vector)
expected_return_GMVP, expected_var_GMVP, expected_volat_GMVP = compute_metrics_ptf(weights_GMVP, returns_array, covariance_matrix)

expected_return : -0.054498
expected_variance : 0.000002
expected_volatility : 0.001400


In [69]:
# Compute efficient portfolio for a given target return
target_return = 0.05

weights_target_ptf = compute_efficient_portfolio_weights(inv_covariance_matrix, unit_vector, returns_vector, target_return)
expected_return_target, expected_var_target, expected_volat_target = compute_metrics_ptf(weights_target_ptf, returns_array, covariance_matrix)

expected_return : 0.050000
expected_variance : 0.000002
expected_volatility : 0.001510
